### CollaborativeFiltering - Test both NLP MultinomialNB model and KNN model

In [1]:
# import required libraries
import pickle
import numpy as np
import pandas as pd
import math
#import vaex

In [16]:
# Load movie_user_rating_index.csv 
movie_features_index_df=pd.read_csv("movie_user_rating_index.csv",encoding ='utf-8',compression='gzip')
movie_features_index_df.head(10)


,index,title
0,0,10 cloverfield lane
1,1,12 years a slave
2,2,22 jump street
3,3,about time
4,4,american hustle
5,5,american sniper
6,6,ant-man
7,7,arrival
8,8,avengers: age of ultron
9,9,baby driver


In [3]:
# by mistaken ly NAN value saved from previous pipline Collabortive filtering model creation
# No issues eventhough with NAN 
movie_features_index_df.isnull().sum()

index    0
title    0
dtype: int64

In [4]:
# Load pivot test table  - used 10 movies data for testing
movie_user_rating_pivottable=pd.read_csv("movie_user_rating_pivottable_test.csv",encoding ='utf-8',compression='gzip')

In [5]:
movie_user_rating_pivottable.head(5)

,14,48,56,72,73,81,119,134,173,176,...,283000,283016,283095,283117,283125,283131,283164,283183,283184,283195
0,0.0,0.0,0.0,0.0,0.0,0.0,0,3.5,3.5,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,4.0,0.0,0.0,4.0,0.0,0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.5,0,3.0,3.0,0.0,...,4.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.0,0.0,0.0,0.0,0.0,0,4.5,5.0,0.0,...,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,3.5,...,4.0,0.0,4.5,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Load KNN model
model_knn = pickle.load( open( "collaborativeFiltering_model.pkl", "rb" ) )

In [7]:
# Load classifier
model_classifier = pickle.load( open( "collaborativeFiltering_NLP_model.pkl", "rb" ) )

In [8]:
# Load TFIDF NLP vectorizer
vectorizer = pickle.load(open("collaborativeFiltering_NLP_vectorizer.pkl", 'rb'))  

In [9]:
# pass movie name and get movie index which can be used in recommendation modelmodel_knn
def getMovieIndex(name):
    name=name.strip().lower()
    firstValue="10 cloverfield lane".lower()
    test_vector= vectorizer.transform([name])
    predicted = model_classifier.predict(test_vector)
    
    available =   name in firstValue
    # return -1 for unavailable movies
    if((predicted[0] == 0) & (~available)):
        return -1
    else:
        return predicted[0]

In [10]:
# Pass index and get similar movies
def getRecomendedMoviesByIndex(queryIndex,numOfMovies):
    distances, indices = model_knn.kneighbors(movie_user_rating_pivottable.iloc[queryIndex,:].values.reshape(1, -1), n_neighbors = numOfMovies)
    return distances,indices

In [11]:
# Pass movie name and get similar movies
def recommendedMovies(name,numOfMovies=5):
    movieList=list()
    query_index=getMovieIndex(name)
    distances, indices =getRecomendedMoviesByIndex(query_index,numOfMovies)
    for i in range(0, len(distances.flatten())):
        movieList.append(movie_features_index_df.title[indices.flatten()[i]])
    return movieList


In [12]:
#name="Iron Man"
#name="101 dalmatians"
name="sflasjlfkjlafl"
numOfMovies=10
getMovieIndex(name)

-1

In [13]:
getRecomendedMoviesByIndex(4,10)

(array([[4.74065232e-14, 4.90238155e-01, 5.16519784e-01, 5.19983278e-01,
         5.35022848e-01, 5.43420639e-01, 5.45842381e-01, 5.47970486e-01,
         5.52988193e-01, 5.56123416e-01]]),
 array([[  4, 104,  21,  38,  39,  44,  40,  18,  63,  12]], dtype=int64))

In [14]:
firstValue="burb"
name="burb"

available = firstValue in name
available

True

In [15]:
name="17 again"
#name="101 dalmatians"
#name="asljflasjflaj sjdflajf"
query_index=getMovieIndex(name)
numOfMovies=10
try:
    distances, indices =getRecomendedMoviesByIndex(query_index,numOfMovies)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_features_index_df.title[query_index]))
        else:
            print('{0}: {1} =  {2}% '.format(i, movie_features_index_df.title[indices.flatten()[i]],(math.ceil((1 - distances.flatten()[i])*100)) ))
except IndexError:
    print("Movie Index not found")
except KeyError:
    print("Movie Index not found")

Movie Index not found
